![PyData_logo](./static/pydata-logo-madrid-2016.png)

# Remove Before Flight
## Analyzing Flight Safety Data with Python

###### Jesús Martos Carrizo
###### Alejandro Sáez Mollejo

### Introduction

#### Some figures 

![passengers](./static/passenger.jpg)
###### by Ryan McGuire

####  Flight Safety

#### Obejctive of this talk 

##### Disclaimer

#### Some remarks about

![pandas-logo](./static/pandas_logo.png)

#### Data source

<img src="./static/ntsb.jpg" style="width: 150px;"/>

__National Transportation Safety Board__ is charged with determining the probable cause of transportation accidents and promoting transportation safety, and assisting victims of transportation accidents and their families.

Data: http://www.ntsb.gov/_layouts/ntsb.aviation/index.aspx

### Loading & Cleaning data 

In [1]:
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# Some configuration options:
pd.set_option('max_columns', 70)
pd.set_option('max_rows', 50)

# to show all posiible options:
# pd.describe_option()

# to reset to default option value:
# pd.reset_option('max_columns')

# to show current value:
# pd.get_option('max_columns')

In [3]:
events = pd.read_csv("./data/events.csv",
                                    sep='|',
                                    index_col=0)  # Use eventid as index
occurrences = pd.read_csv("./data/Occurrences1.csv",
                                    sep=';',
                                    index_col=0)  # Use eventid as index

/miniconda/envs/pydata-safety/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##### A first glipse of the data 

In [4]:
events.head(5)
# there is an extra row for ev_id with no data because we have used index_col=0 as an argument in the function read_csv()

,ntsb_no,ev_type,ev_date,ev_dow,ev_time,ev_tmzn,ev_city,ev_state,ev_country,ev_site_zipcode,ev_year,ev_month,mid_air,on_ground_collision,latitude,longitude,latlong_acq,apt_name,ev_nr_apt_id,ev_nr_apt_loc,apt_dist,apt_dir,apt_elev,wx_brief_comp,wx_src_iic,wx_obs_time,wx_obs_dir,wx_obs_fac_id,wx_obs_elev,wx_obs_dist,wx_obs_tmzn,light_cond,sky_cond_nonceil,sky_nonceil_ht,sky_ceil_ht,sky_cond_ceil,vis_rvr,vis_rvv,vis_sm,wx_temp,wx_dew_pt,wind_dir_deg,wind_dir_ind,wind_vel_kts,wind_vel_ind,gust_ind,gust_kts,altimeter,wx_dens_alt,wx_int_precip,metar,ev_highest_injury,inj_f_grnd,inj_m_grnd,inj_s_grnd,inj_tot_f,inj_tot_m,inj_tot_n,inj_tot_s,inj_tot_t,invest_agy,ntsb_docket,ntsb_notf_from,ntsb_notf_date,ntsb_notf_tm,fiche_number,lchg_date,lchg_userid,wx_cond_basic,faa_dist_office
ev_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20001208X07734,LAX97FA143,ACC,04/02/1997 12:00:00 AM,WE,2016.0,MST,GRAND CANYON,AZ,USA,86023,1997.0,4,N,N,,,NaN,,,OFAP,NaN,NaN,NaN,PARP,WFAC,1954.0,180.0,GCN,6606.0,21.0,MST,NDRK,SCAT,3600.0,6000.0,BKN,"0,00000000e+00",0.0,"1,00000000e+01",-2.0,-3.0,0.0,U,0.0,CALM,N,0.0,"2,90000000e+01",NaN,UNK,NaN,FATL,0.0,0.0,0.0,2.0,NaN,NaN,NaN,2.0,N,1890.0,NaN,04/02/1997 12:00:00 AM,1922.0,IMAGE,12/08/2000 12:13:19 PM,dbo,IMC,NaN
20021008X05297,CHI02LA299,ACC,09/29/2002 12:00:00 AM,Su,2100.0,CDT,Moorhead,MN,USA,56560,2002.0,9,N,N,465021N,0963947W,NaN,Moorhead Municipal,JKJ,ONAP,NaN,NaN,917.0,UNK,WFAC,2053.0,47.0,FAR,908.0,9.0,BST,NDRK,CLER,NaN,1300.0,OVC,NaN,NaN,"7,00000000e+00",14.0,12.0,160.0,Y,13.0,SPEC,Y,0.0,"2,97099991e+01",NaN,NaN,NaN,SERS,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,N,13246.0,FAA,10/03/2002 12:00:00 AM,945.0,DMS,04/23/2003 12:26:44 PM,NTSB\\MONR,VMC,Minneapolis FSDO
20040127X00111,LAX04LA103,ACC,10/12/2003 12:00:00 AM,Su,1640.0,MST,Scottsdale,AZ,USA,85255,2003.0,10,N,N,333722N,1115438W,NaN,Scottsdale,SDL,ONAP,NaN,NaN,1510.0,FULL,WFAC,1653.0,NaN,KSDL,1510.0,NaN,MST,DAYL,CLER,NaN,NaN,NONE,NaN,NaN,"1,00000000e+01",33.0,12.0,NaN,NaN,NaN,CALM,N,NaN,"2,98899994e+01",NaN,NaN,NaN,NONE,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,N,17586.0,FAA,01/22/2004 12:00:00 AM,1215.0,DMS,04/28/2005 02:59:26 PM,NTSB\\JOHB,VMC,WP-SDL-FSDO
20001213X34369,ANC86LA135,ACC,08/23/1986 12:00:00 AM,SA,1100.0,AKD,TALKEETNA,AK,USA,99676,1986.0,8,N,N,,,NaN,,,OFAP,"0,00000000e+00",0.0,0.0,UNK,PILO,0.0,0.0,,0.0,0.0,,DAYL,CLER,0.0,0.0,NONE,"0,00000000e+00",0.0,"5,00000000e+01",10.0,-18.0,0.0,U,0.0,UNK,U,0.0,NaN,NaN,UNK,NaN,NONE,0.0,0.0,0.0,NaN,NaN,3.0,NaN,NaN,N,891.0,NaN,08/23/1986 12:00:00 AM,1430.0,31238,01/02/2001 09:14:50 AM,dbo,VMC,NaN
20001213X35290,ANC87LA019,ACC,12/07/1986 12:00:00 AM,SU,1256.0,AST,SHAGELUK,AK,USA,99665,1986.0,12,N,N,,,NaN,,1,OFAP,"0,00000000e+00",0.0,0.0,UNK,PILO,0.0,0.0,,0.0,0.0,,DAYL,SCAT,1500.0,0.0,NONE,"0,00000000e+00",0.0,"4,00000000e+01",-1.0,-1.0,0.0,U,0.0,CALM,N,0.0,"2,90000000e+01",NaN,LGT,NaN,NONE,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,N,1400.0,NaN,12/07/1986 12:00:00 AM,1300.0,31791,01/02/2001 09:14:55 AM,dbo,VMC,NaN


In [5]:
events.columns

Index(['ntsb_no', 'ev_type', 'ev_date', 'ev_dow', 'ev_time', 'ev_tmzn',
       'ev_city', 'ev_state', 'ev_country', 'ev_site_zipcode', 'ev_year',
       'ev_month', 'mid_air', 'on_ground_collision', 'latitude', 'longitude',
       'latlong_acq', 'apt_name', 'ev_nr_apt_id', 'ev_nr_apt_loc', 'apt_dist',
       'apt_dir', 'apt_elev', 'wx_brief_comp', 'wx_src_iic', 'wx_obs_time',
       'wx_obs_dir', 'wx_obs_fac_id', 'wx_obs_elev', 'wx_obs_dist',
       'wx_obs_tmzn', 'light_cond', 'sky_cond_nonceil', 'sky_nonceil_ht',
       'sky_ceil_ht', 'sky_cond_ceil', 'vis_rvr', 'vis_rvv', 'vis_sm',
       'wx_temp', 'wx_dew_pt', 'wind_dir_deg', 'wind_dir_ind', 'wind_vel_kts',
       'wind_vel_ind', 'gust_ind', 'gust_kts', 'altimeter', 'wx_dens_alt',
       'wx_int_precip', 'metar', 'ev_highest_injury', 'inj_f_grnd',
       'inj_m_grnd', 'inj_s_grnd', 'inj_tot_f', 'inj_tot_m', 'inj_tot_n',
       'inj_tot_s', 'inj_tot_t', 'invest_agy', 'ntsb_docket', 'ntsb_notf_from',
       'ntsb_notf_date', 'ntsb_n

In [6]:
# Why should we change the column names?
# Moreover, names with capitals and spaces make it more complicated for us
# to select the column (ie. `events.ev_dow` is valid and `events.Event day of the week` is not)

#events = events.rename(columns = {'ev_dow':'Event day of the week','light_cond':'Light Conditions','wind_vel_kts':'Wind Speed(knots)',
#                       'metar':'METAR Weather Report','vis_rvr':'Visibility Runway Visual Range (Feet)',
#                        'vis_rvv':'Visibility Runway Visual Value (Statute Miles)','vis_sm':'Visibility (Statute Miles)',
#                        'wx_cond_basic':'Basic Weather Conditions'})

In [7]:
events.dtypes

ntsb_no                 object
ev_type                 object
ev_date                 object
ev_dow                  object
ev_time                float64
ev_tmzn                 object
ev_city                 object
ev_state                object
ev_country              object
ev_site_zipcode         object
ev_year                float64
ev_month                object
mid_air                 object
on_ground_collision     object
latitude                object
longitude               object
latlong_acq             object
apt_name                object
ev_nr_apt_id            object
ev_nr_apt_loc           object
apt_dist                object
apt_dir                float64
apt_elev               float64
wx_brief_comp           object
wx_src_iic              object
                        ...   
gust_ind                object
gust_kts               float64
altimeter               object
wx_dens_alt            float64
wx_int_precip           object
metar                   object
ev_highe

In [8]:
# Not really useful for the majority of columns
events.describe()

,ev_time,ev_year,apt_dir,apt_elev,wx_obs_time,wx_obs_dir,wx_obs_elev,wx_obs_dist,sky_nonceil_ht,sky_ceil_ht,vis_rvv,wx_temp,wx_dew_pt,wind_dir_deg,wind_vel_kts,gust_kts,wx_dens_alt,inj_f_grnd,inj_m_grnd,inj_s_grnd,inj_tot_f,inj_tot_m,inj_tot_n,inj_tot_s,inj_tot_t,ntsb_docket,ntsb_notf_tm
count,76535.000000,76848.000000,37020.000000,48146.000000,69909.000000,63990.000000,67552.000000,65299.000000,56454.000000,55382.000000,48754.000000,67043.000000,55773.000000,67986.000000,69080.000000,53729.000000,20314.000000,49522.000000,49526.000000,49513.000000,15253.000000,14347.000000,46273.000000,10793.000000,35217.000000,72695.000000,56995.000000
mean,1373.671196,1996.920427,49.827769,1055.010344,999.464661,103.528911,866.237728,11.339745,3255.918819,2930.506861,0.146039,19.850603,6.877970,161.884167,7.257426,3.650245,1772.910357,0.010157,0.013064,0.011209,2.813545,1.866732,7.574676,1.543037,2.451969,8586.409781,1043.181577
std,434.386422,101.770110,100.047186,1655.995540,720.507141,122.243560,3746.138486,30.473635,6264.894555,6349.779335,22.881972,17.341558,15.660181,117.043219,5.951389,10.485619,2706.629533,0.353946,0.393807,0.518030,11.025730,5.172820,31.982420,2.722736,8.733082,10805.425263,718.420703
min,0.000000,1948.000000,0.000000,-210.000000,0.000000,0.000000,-115.000000,0.000000,0.000000,0.000000,0.000000,-62.000000,-38.000000,0.000000,0.000000,0.000000,-30536.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1100.000000,1988.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,45.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,945.000000,1.000000
50%,1400.000000,1996.000000,0.000000,434.000000,1153.000000,40.000000,128.000000,1.000000,0.000000,0.000000,0.000000,21.000000,8.000000,170.000000,7.000000,0.000000,500.000000,0.000000,0.000000,0.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2005.000000,1200.000000
75%,1700.000000,2005.000000,20.000000,1071.000000,1553.000000,200.000000,845.000000,16.000000,4000.000000,2700.000000,0.000000,27.000000,16.000000,260.000000,10.000000,0.000000,2370.750000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000,16468.500000,1600.000000
max,2400.000000,26733.000000,979.000000,13250.000000,2400.000000,360.000000,751635.000000,1526.000000,250000.000000,250000.000000,4444.000000,1652.000000,1646.000000,999.000000,210.000000,999.000000,26858.000000,44.000000,62.000000,71.000000,349.000000,380.000000,699.000000,111.000000,380.000000,34935.000000,2400.000000


In [9]:
# Lets gather some info about the data
events.info(max_cols=0)

<class 'pandas.core.frame.DataFrame'>
Index: 76880 entries, 20001208X07734 to 20150121X60411
Columns: 70 entries, ntsb_no to faa_dist_office
dtypes: float64(27), object(43)
memory usage: 41.6+ MB


In [10]:
# What does 41.6+ MB mean???
events.info(max_cols=0, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 76880 entries, 20001208X07734 to 20150121X60411
Columns: 70 entries, ntsb_no to faa_dist_office
dtypes: float64(27), object(43)
memory usage: 209.2 MB


In [11]:
events.memory_usage(deep=True).sum() / 1048  # KiB  

209329.80534351146

In [12]:
# with deep=False same as events.values.nbyte; what is the difference when is True?
events.values.nbytes / 1048

41080.916030534354

In [13]:
events.info(max_cols=0, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 76880 entries, 20001208X07734 to 20150121X60411
Columns: 70 entries, ntsb_no to faa_dist_office
dtypes: float64(27), object(43)
memory usage: 209.2 MB


##### Selecting only certain events 

##### Accidents 

In [14]:
# Maybe there is a better way to select...
# Make a brief comparison of results and performance (%timeit)
events = events[events['ev_type'] == 'ACC']
# options:
# events = events.where(events['ev_dow']=='WE')       # the return a DataFrame of the same shape as the original.When the logical 
                                                    # condition in False the result is the whole row filled with NaNs.
                                                    # By default, where returns a modified copy of the data. There is an 
                                                    # optional parameter inplace so that the original data can be modified
                                                    # without creating a copy

# events = events.where(events['ev_dow']=='WE', 0, inplace=True)     # same that above but choosing the result instead of NaN and 
                                                                 # without creating a copy

# events = events.mask(events['ev_dow']=='WE')      # mask is the inverse boolean operation of where

# events = events.query('ev_type == ACC')     #  query() method that allows selection using an expression. In pandas 0.18 is 
                                              # an experimental method. Can this become useful?

##### Kind of flight 

In [15]:
most_victims = events['inj_tot_t'].sort_values(ascending=False).iloc[0:150]
most_victims.head(10)

ev_id
20020124X00116    349.0
20140718X92314    295.0
20001213X28786    283.0
20001213X27403    275.0
20011130X02321    265.0
20001214X38384    256.0
20001208X08606    254.0
20140308X91420    239.0
20001211X16146    231.0
20001208X06204    230.0
Name: inj_tot_t, dtype: float64

In [16]:
most_victims.count()

150

In [17]:
aircraft = pd.read_csv("./data/aircraft.csv",
                                    encoding = "ISO-8859-1",  # export the table again.
                                    sep=';')

# check DtypteWarning 
# Columns (4,7,17,27,28,32,37,38,47,48,53,63,70,72,73,76,82) have mixed types.
# Specify dtype option on import or set low_memory=False.

/miniconda/envs/pydata-safety/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4,7,17,27,28,32,37,38,47,48,53,63,70,72,73,76,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
aircrafts_most_victims = aircraft[aircraft['ev_id'].isin(most_victims.index)]
aircrafts_most_victims

,ev_id,Aircraft_Key,regis_no,ntsb_no,acft_missing,far_part,flt_plan_filed,flight_plan_activated,damage,acft_fire,acft_expl,acft_make,acft_model,acft_series,acft_serial_no,cert_max_gr_wt,acft_category,acft_reg_cls,homebuilt,fc_seats,cc_seats,pax_seats,total_seats,num_eng,fixed_retractable,type_last_insp,date_last_insp,afm_hrs_last_insp,afm_hrs,elt_install,elt_oper,elt_aided_loc_ev,elt_type,owner_acft,owner_street,...,second_pilot,dprt_pt_same_ev,dprt_apt_id,dprt_city,dprt_state,dprt_country,dprt_time,dprt_timezn,dest_same_local,dest_apt_id,dest_city,dest_state,dest_country,phase_flt_spec,report_to_icao,evacuation,lchg_date,lchg_userid,afm_hrs_since,rwy_num,rwy_len,rwy_width,site_seeing,air_medical,med_type_flight,acft_year,fuel_on_board,commercial_space_flight,unmanned,ifr_equipped_cert,elt_mounted_aircraft,elt_connected_antenna,elt_manufacturer,elt_model,elt_reason_other
1721,20001206X01727,1,N954VJ,DCA94MA065,N,121,IFR,NaN,DEST,GRD,NONE,DOUGLAS,DC-9-31,DC-9-31,47590,105000.0,AIR,USUS,N,NaN,NaN,NaN,103.0,2.0,RETR,COAW,01/07/1994,NaN,53917,N,U,U,NaN,USAIR,NaN,...,Y,N,CAE,COLUMBIA,SC,NaN,1823.0,EDT,SAME,CLT,NaN,NaN,NaN,560.0,NaN,NaN,20/12/2002,dbo,NaN,18R,10000.0,150.0,N,N,NaN,NaN,NaN,FALSO,FALSO,FALSO,FALSO,FALSO,NaN,NaN,NaN
2229,20001206X02233,1,N513AU,DCA94MA076,N,121,IFR,NaN,DEST,GRD,GRD,BOEING,B-737-300,B-737-300,24549,142500.0,AIR,USUS,N,NaN,NaN,NaN,142.0,2.0,RETR,COAW,08/08/1994,274,23846,N,U,U,NaN,USAIR,2345 CRYSTAL DRIVE,...,Y,N,ORD,CHICAGO,IL,NaN,1830.0,CDT,NaN,PIT,PITTSBURG,PA,NaN,560.0,NaN,NaN,02/01/2001,dbo,NaN,0,NaN,NaN,N,N,NaN,NaN,NaN,FALSO,FALSO,FALSO,FALSO,FALSO,NaN,NaN,NaN
2418,20001206X02420,1,N401AM,DCA95MA001,N,121,IFR,Y,DEST,NONE,NONE,ATR,ATR-72-212,ATR-72-212,401,47400.0,AIR,USUS,N,NaN,NaN,NaN,68.0,2.0,RETR,COAW,29/03/1994,NaN,1352,N,U,U,NaN,"AMR Group, Inc.",NaN,...,Y,N,IND,INDIANAPOLIS,IN,NaN,1455.0,CST,NaN,ORD,CHICAGO,IL,NaN,0.0,NaN,NaN,16/05/2002,NTSB\HAMG,NaN,0,NaN,NaN,N,N,NaN,NaN,NaN,FALSO,FALSO,FALSO,FALSO,FALSO,NaN,NaN,NaN
2710,20001206X02711,1,NaN,DCA95WA012,N,NUSC,IFR,NaN,DEST,UNK,UNK,BOEING,737-400,737-400,NaN,NaN,AIR,UNK,N,NaN,NaN,NaN,NaN,NaN,NaN,UNK,NaN,NaN,NaN,U,U,U,NaN,NaN,NaN,...,U,N,NaN,"ANKARA, TURKEY",OF,NaN,0.0,NaN,SAME,NaN,NaN,NaN,NaN,0.0,NaN,NaN,22/10/2001,dbo,NaN,0,NaN,NaN,N,N,NaN,NaN,NaN,FALSO,FALSO,FALSO,FALSO,FALSO,NaN,NaN,NaN
2848,20001207X02846,1,NaN,DCA95RA013,N,NUSC,IFR,NaN,DEST,UNK,UNK,McDonnell Douglas,DC-9-14,DC-9-14,NaN,NaN,AIR,UNK,N,NaN,NaN,NaN,NaN,NaN,NaN,UNK,NaN,NaN,NaN,U,U,U,NaN,NaN,NaN,...,U,N,BOG,BOGOTA,OF,NaN,0.0,NaN,NaN,CTG,CARTAGENA,OF,NaN,0.0,NaN,NaN,22/10/2001,dbo,NaN,0,NaN,NaN,N,N,NaN,NaN,NaN,FALSO,FALSO,FALSO,FALSO,FALSO,NaN,NaN,NaN
3371,20001207X03362,1,N1768D,NYC95LA083,N,121,IFR,NaN,NONE,GRD,NONE,McDonnell Douglas,MD-11,MD-11,48436,621000.0,AIR,USUS,N,NaN,NaN,NaN,287.0,3.0,RETR,COAW,NaN,NaN,11617,N,U,U,NaN,AMERICAN AIRLINES,DALLAS - FT. WORTH AIRPORT,...,Y,Y,NaN,NaN,NaN,NaN,0.0,NaN,NaN,SDQ,SANTO DOMINGO,OF,NaN,510.0,NaN,NaN,02/01/2001,dbo,NaN,0,NaN,NaN,N,N,NaN,NaN,NaN,FALSO,FALSO,FALSO,FALSO,FALSO,NaN,NaN,NaN
4248,20001207X04224,1,N125GU,DCA95RA051,N,129,IFR,NaN,DEST,UNK,UNK,Boeing,737-200,737-200,NaN,NaN,AIR,UNK,N,NaN,NaN,NaN,NaN,NaN,NaN,UNK,NaN,NaN,NaN,U,U,U,NaN,NaN,NaN,...,U,N,MGGT,GUATEMALA CITY,OF,NaN,0.0,NaN,NaN,MSLP,SAN SALVADOR,OF,NaN,0.0,NaN,NaN,03/01/2001,dbo,NaN,0,NaN,NaN,N,N,NaN,NaN,NaN,FALSO,FALSO,FALSO,FALSO,FALSO,NaN,NaN,NaN
5018,20001207X04989,1,NaN,DCA96RA014,N,NUSC,IFR,NaN,DEST,UNK,UNK,BOEING,B-737-200,B-737-200,NaN,NaN,AIR,UNK,N,NaN,NaN,NaN,NaN,NaN,NaN,UNK,NaN,NaN,NaN,U,U,U,NaN,NaN,NaN,...,U,N,NaN,"COTONOU,BENIN",OF,NaN,0.0,NaN,SAME,NaN,NaN,NaN,NaN,0.0,NaN,NaN,22/10/2001,dbo,NaN,0,NaN,NaN,N,N,NaN,NaN,NaN,FALSO,FALSO,FALSO,FALSO,FALSO,NaN,NaN,NaN
5019,20001207X04990,1,N651AA,DCA96RA020,N,121,IFR,NaN,DEST,UNK,UNK,Boeing,B757,B757,NaN,NaN,AIR,UNK,N,NaN,NaN,NaN,NaN,NaN,RETR,UNK,NaN,NaN,NaN,U,U,U,NaN,NaN,NaN,...,U,N,MIA,MIAMI,FL,NaN,0.0,NaN,NaN,NaN,CALI,OF,NaN,560.0,NaN,NaN,03/01/2001,dbo,NaN,0,NaN,NaN,N,N,NaN,NaN,NaN,FALSO,FALSO,FALSO,FALSO,F

In [19]:
# Number of accidents in each far part
aircrafts_most_victims['far_part'].value_counts()

NUSC    62
121     49
129     33
091      4
NUSN     2
UNK      2
135      1
Name: far_part, dtype: int64

In [20]:
# Aircraft make
aircrafts_most_victims['acft_make'].value_counts() 

Boeing                          49
BOEING                          33
McDonnell Douglas               15
Airbus Industrie                 8
MCDONNELL DOUGLAS                7
AIRBUS                           6
ATR                              4
Fokker                           4
DOUGLAS                          4
Douglas                          3
LOCKHEED                         3
Tupolev                          2
Bombardier, Inc.                 1
FAIRCHILD                        1
Lockheed                         1
NORTH AMERICAN/AERO CLASSICS     1
AIRBUS INDUSTRIE                 1
BRITISH AEROSPACE                1
EMBRAER                          1
Bombardier                       1
British Aerospace                1
BEECH                            1
CASA                             1
Aerospatiale                     1
Canadair                         1
Convair                          1
PIPER                            1
Embraer                          1
BOMBARDIER INC      

In [21]:
# Some of them are in caps
aircrafts_most_victims['acft_make'] = aircrafts_most_victims['acft_make'].str.lower()
aircrafts_most_victims['acft_make'].value_counts() 

/miniconda/envs/pydata-safety/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


boeing                          82
mcdonnell douglas               22
airbus industrie                 9
douglas                          7
airbus                           6
fokker                           5
atr                              4
lockheed                         4
tupolev                          3
british aerospace                2
embraer                          2
bombardier, inc.                 1
fairchild                        1
north american/aero classics     1
piper                            1
casa                             1
bombardier                       1
convair                          1
aerospatiale                     1
canadair                         1
beech                            1
cessna                           1
bombardier inc                   1
Name: acft_make, dtype: int64

In [22]:
# Far part
# Check again!
desired_far_parts = ['NUSC',  # Non-U.S. Commercial
                                 '121',      # Air Carrier
                                 '129',      # Foreign
                              # '091',      # General Aviation
                                 'NUSN',  # Non-U.S. Non-Commercial
                                 'UNK',     # Unknown
                                 '135']      # Air Taxi & Commuter

In [23]:
cond = aircraft['far_part'].isin(desired_far_parts)
ev_ids_for_desired_far = aircraft[cond]['ev_id'].drop_duplicates()

ev_ids_for_desired_far.count()

8513

In [24]:
events = events.loc[ev_ids_for_desired_far]
events['ev_type'].count()

6458

##### Converting types 

In [25]:
events['ev_date'].head()

ev_id
20001204X00000    01/10/1999 12:00:00 AM
20001204X00001                       NaN
20001204X00002    01/05/1999 12:00:00 AM
20001204X00004    01/12/1999 12:00:00 AM
20001204X00020    01/06/1999 12:00:00 AM
Name: ev_date, dtype: object

In [26]:
#%timeit pd.to_datetime(events['ev_date'], infer_datetime_format=True)

In [27]:
#%timeit pd.to_datetime(events['ev_date'], infer_datetime_format=False)

In [28]:
# Information about date format: 
# https://docs.python.org/3.5/library/datetime.html#strftime-and-strptime-behavior
date_format = "%m/%d/%Y %I:%M:%S %p"
% timeit pd.to_datetime(events['ev_date'], format=date_format)

10 loops, best of 3: 69.7 ms per loop


In [29]:
date_format = "%m/%d/%Y %I:%M:%S %p"
events['ev_date'] = pd.to_datetime(events['ev_date'], format=date_format)

In [30]:
events['ev_date'][0].dayofweek

6

In [31]:
# Convert column dtype to string, extract the time and pad with zeros
ev_time = events['ev_time'].astype(str).str.split('.').str[0].str.zfill(4)

pd.to_datetime(ev_time,
               format="%H%M",  # format
               errors='coerce')  # for nan values

ev_id
20001204X00000   1900-01-01 13:30:00
20001204X00001                   NaT
20001204X00002   1900-01-01 14:25:00
20001204X00004   1900-01-01 10:50:00
20001204X00020   1900-01-01 13:15:00
20001204X00035   1900-01-01 23:18:00
20001204X00038   1900-01-01 00:00:00
20001204X00044   1900-01-01 06:11:00
20001204X00045   1900-01-01 20:40:00
20001204X00047                   NaT
20001204X00066   1900-01-01 19:30:00
20001204X00072                   NaT
20001204X00087                   NaT
20001204X00105   1900-01-01 11:20:00
20001204X00109   1900-01-01 16:55:00
20001204X00114                   NaT
20001205X00118   1900-01-01 23:45:00
20001205X00119                   NaT
20001205X00124   1900-01-01 18:41:00
20001205X00131   1900-01-01 11:35:00
20001205X00145   1900-01-01 22:46:00
20001205X00146                   NaT
20001205X00147                   NaT
20001205X00148   1900-01-01 17:11:00
20001205X00149   1900-01-01 08:27:00
                         ...        
20151216X41606   1900-01-01 17:3

In [32]:
events['ev_time'] = pd.to_datetime(ev_time, format="%H%M", errors='coerce') 

In [33]:
events['ev_date'][0:5].dt.date.astype(str) + ' ' + events['ev_time'][0:5].dt.time.astype(str)

ev_id
20001204X00000    1999-01-10 13:30:00
20001204X00001                nan nan
20001204X00002    1999-01-05 14:25:00
20001204X00004    1999-01-12 10:50:00
20001204X00020    1999-01-06 13:15:00
dtype: object

In [34]:
# FIXME
time_series = pd.to_datetime(events['ev_date'].dt.date.astype(str) + ' ' + events['ev_time'].dt.time.astype(str), 
                         format="%Y-%m-%d %H:%M:%S",
                         errors='coerce')  # !! WATCH OUT I am loosing some dates due to missing times...

In [35]:
pd.to_datetime('2002-09-29' + ' ' + 'NaT', 
                               format="%Y-%m-%d %H:%M:%S",
                               errors='coerce')

NaT

In [36]:
pd.to_datetime('2002-09-29' + ' ' + '00', 
                               format="%Y-%m-%d %H:%M:%S",
                               errors='coerce')

Timestamp('2002-09-29 00:00:00')

In [37]:
# isnull detects 'NaT'
pd.isnull(events['ev_time']).sum()

2172

In [38]:
# Aha! so the same...
pd.isnull(time_series).sum()

2172

In [39]:
events.sort_values('inj_tot_t', ascending=False)

,ntsb_no,ev_type,ev_date,ev_dow,ev_time,ev_tmzn,ev_city,ev_state,ev_country,ev_site_zipcode,ev_year,ev_month,mid_air,on_ground_collision,latitude,longitude,latlong_acq,apt_name,ev_nr_apt_id,ev_nr_apt_loc,apt_dist,apt_dir,apt_elev,wx_brief_comp,wx_src_iic,wx_obs_time,wx_obs_dir,wx_obs_fac_id,wx_obs_elev,wx_obs_dist,wx_obs_tmzn,light_cond,sky_cond_nonceil,sky_nonceil_ht,sky_ceil_ht,sky_cond_ceil,vis_rvr,vis_rvv,vis_sm,wx_temp,wx_dew_pt,wind_dir_deg,wind_dir_ind,wind_vel_kts,wind_vel_ind,gust_ind,gust_kts,altimeter,wx_dens_alt,wx_int_precip,metar,ev_highest_injury,inj_f_grnd,inj_m_grnd,inj_s_grnd,inj_tot_f,inj_tot_m,inj_tot_n,inj_tot_s,inj_tot_t,invest_agy,ntsb_docket,ntsb_notf_from,ntsb_notf_date,ntsb_notf_tm,fiche_number,lchg_date,lchg_userid,wx_cond_basic,faa_dist_office
ev_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20020124X00116,DCA97WA007,ACC,1996-11-12,Tu,1900-01-01 18:40:00,NaN,New Delhi,NaN,IN,NaN,1996.0,11,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FATL,NaN,NaN,NaN,349.0,NaN,NaN,NaN,349.0,O,NaN,NaN,NaN,NaN,NaN,01/24/2002 01:55:56 PM,HAUT,NaN,NaN
20140718X92314,DCA14RA127,ACC,2014-07-17,Th,NaT,NaN,Hrabove,NaN,UP,NaN,2014.0,7,N,N,NaN,NaN,NaN,NaN,NaN,OFAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FATL,NaN,NaN,NaN,295.0,NaN,NaN,NaN,295.0,O,NaN,Ukrainian NBAAI,NaN,NaN,NaN,06/23/2015 03:49:42 PM,sedj,NaN,NaN
20001213X28786,DCA89MA063,ACC,1989-07-19,WE,1900-01-01 16:00:00,CDT,SIOUX CITY,IA,USA,51102,1989.0,7,N,N,,,NaN,SIOUX GATEWAY,SUX,ONAP,"0,00000000e+00",0.0,1098.0,NOTP,UNK,0.0,0.0,SUX,0.0,0.0,,DAYL,UNK,0.0,4000.0,BKN,"0,00000000e+00",0.0,"1,50000000e+01",27.0,14.0,10.0,U,11.0,UNK,U,0.0,"3,00000000e+01",0.0,UNK,NaN,FATL,0.0,0.0,0.0,111.0,125.0,13.0,47.0,283.0,N,437.0,NaN,07/19/1989 12:00:00 AM,1710.0,39915,12/13/2000 12:51:26 PM,dbo,VMC,NaN
20001213X27403,DCA89RA014,ACC,1988-12-21,WE,1900-01-01 14:05:00,EST,LOCKERBIE,NaN,UK,,1988.0,12,N,N,,,NaN,,,UNK,"0,00000000e+00",0.0,0.0,UNK,UNK,0.0,0.0,,0.0,0.0,,NR,UNK,0.0,0.0,UNK,"0,00000000e+00",0.0,"0,00000000e+00",-18.0,-18.0,0.0,U,0.0,UNK,U,0.0,NaN,0.0,UNK,NaN,FATL,11.0,3.0,2.0,270.0,3.0,NaN,2.0,275.0,N,7000.0,NaN,12/21/1988 12:00:00 AM,1500.0,41026,12/18/2002 11:28:47 AM,NTSB\\GIUV,UNK,NaN
20011130X02321,DCA02MA001,ACC,2001-11-12,Mo,1900-01-01 09:16:00,EST,Belle Harbor,NY,USA,11694,2001.0,11,N,N,NaN,NaN,NaN,John F. Kennedy International,JFK,OFAP,NaN,NaN,NaN,FULL,WFAC,925.0,NaN,JFK,NaN,NaN,EST,DAYL,FEW,4800.0,NaN,NONE,NaN,NaN,"1,00000000e+01",6.0,-6.0,270.0,Y,8.0,SPEC,N,NaN,"3,04400005e+01",NaN,NaN,NaN,FATL,5.0,NaN,NaN,265.0,NaN,NaN,NaN,265.0,N,17648.0,,NaN,NaN,DMS,04/14/2005 12:22:15 PM,NTSB\\JOHB,VMC,NaN
20001214X38384,DCA86RA010,ACC,1985-12-12,TH,1900-01-01 05:18:00,EST,GANDER,NaN,CA,,1985.0,12,N,N,,,NaN,GANDER INTERNATIONAL,,ONAP,"0,00000000e+00",0.0,NaN,UNK,UNK,0.0,0.0,,0.0,0.0,,DAYL,UNK,0.0,0.0,UNK,"0,00000000e+00",0.0,"0,00000000e+00",-16.0,-18.0,0.0,U,15.0,UNK,U,0.0,NaN,0.0,LGT,NaN,FATL,0.0,0.0,0.0,256.0,NaN,NaN,NaN,256.0,N,7000.0,NaN,12/12/1985 12:00:00 AM,0.0,39507,12/18/2002 11:28:47 AM,NTSB\\GIUV,IMC,NaN
20001208X08606,DCA97MA058,ACC,1997-08-06,We,1900-01-01 01:42:00,YDT,NIMITZ HILL,GU,USA,NaN,1997.0,8,N,N,NaN,NaN,NaN,AGANA INTERNATIONAL AIRPO,KGUM,OFAP,"3,00000000e+00",240.0,297.0,UNK,WFAC,132.0,NaN,GUM,297.0,NaN,NaN,NDRK,SCAT,1600.0,2500.0,BKN,NaN,NaN,"4,00000000e+00",27.0,25.0,90.0,Y,6.0,SPEC,N,NaN,"2,90000000e+01",NaN,LGT,NaN,FATL,0.0,0.0,0.0,228.0,NaN,NaN,26.0,254.0,N,20688.0,NaN,08/05/1997 12:00:00 AM,1200.0,IMAGE,10/23/2009 11:11:47 AM,johb,IMC,NaN
20140308X91420,DCA14RA076,ACC,2014-03-08,Sa,1900-01-01 02:40:00,LCL,Kuala Lampur,NaN,MY,NaN,2014.0,3,N,N,NaN,NaN,EST,Kuala Lumper International,WMKK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FATL,NaN,NaN,NaN,239.0,Na

In [40]:
events.loc[events_far21.drop_duplicates().values].count()

NameError: name 'events_far21' is not defined

In [ ]:
events.loc[events_far21.drop_duplicates().values][['inj_tot_f', 'inj_tot_m', 'inj_tot_s', 'inj_tot_t']].sort_values(
    'inj_tot_t', ascending=False)

In [ ]:
events.count()

In [ ]:
events['inj_tot_f']

In [ ]:
lat = events['latitude'].dropna()
mask = lat.str.contains(r'^[0-9]{5,6}[NnSs]$')
mask

In [ ]:
def convert_lat(string):
    degs = float(string[0:2])
    mins = float(string[2:4])
    secs = float(string[4:6])
    last = string[6].lower()
    if last == 's':
        factor = -1.0
    elif last == 'n':
        factor = 1.0
    else:
        raise ValueError("invalid hemisphere")
    return factor * (degs + mins / 60 + secs / 3600)

In [ ]:
lat = events['latitude'].dropna()
mask = lat.str.contains(r'^[0-9]{6}[NnSs]$')
events['latitude_num'] = lat[mask].apply(convert_lat)
events['latitude_num']
# pd.notnull(events['latitude_num']).count() is the same with 6 and {5,6} in regex

In [ ]:
def convert_lon(string):
    degs = float(string[0:3])
    mins = float(string[3:5])
    secs = float(string[5:7])
    last = string[7].lower()
    if last == 'w':
        factor = -1.0
    elif last == 'e':
        factor = 1.0
    else:
        raise ValueError("invalid direction")
    return factor * (degs + mins / 60 + secs / 3600)

In [ ]:
lon = events['longitude'].dropna()
mask = lon.str.contains(r'^[0-9]{7}[EeWw]$')
events['longitude_num'] = lon[mask].apply(convert_lon)
events['longitude_num']
# pd.notnull(events['longitude_num']).count() is the same with 7 and {6,7} in regex
# and the same as notnull in latitude. Yuhu!

In [ ]:
events.drop(['latitude', 'longitude', 'latlong_acq'], axis=1, inplace=True)

In [ ]:
%matplotlib qt
# miller projection
from mpl_toolkits.basemap import Basemap
map = Basemap('mill',lon_0=0, lat_0=0)
# plot coastlines, draw label meridians and parallels.
map.drawcoastlines()
# fill continents 'coral' (with zorder=0), color wet areas 'aqua'
#map.drawmapboundary(fill_color='aqua')
#map.fillcontinents(color='coral',lake_color='aqua')
map.bluemarble()
map.scatter(events['longitude_num'].values, events['latitude_num'].values, latlon=True, marker='o', color='r')

### Data Analysis 

#### Accidents location 

#### Accident trends 

#### When do accidents occur? 

In [ ]:
a = occurrences['Phase_of_Flight'].value_counts()
b = a[0:7]  #  Why are we discarding the rest of maneuvers?

maneuvers = {572:'LANDING-ROLL',
             571:'LANDING - FLARE/TOUCHDOWN',
             553:'DESCENT - UNCONTROLLED',
             552:'DESCENT - EMERGENCY (plt initiated)',
             522:'TAKEOFF - INITIAL CLIMB',
             580:'MANEUVERING (includes buzzing)',
             540:'CRUISE (includes low altitude straight and level flight)'}

b = b.rename_axis(maneuvers)

b

#### Other Questions... 

### Conclusions 

# Thanks for yor attention! 

![PyData_logo](./static/pydata-logo-madrid-2016.png)

## Any Questions?


---


In [ ]:
# Notebook style
from IPython.core.display import HTML
css_file = './static/style.css'
HTML(open(css_file, "r").read())